In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
%alias nbconvert nbconvert ./slimpi_graceful.ipynb

In [150]:
%nbconvert

[NbConvertApp] Converting notebook ./slimpi_graceful.ipynb to python


In [4]:
import logging
import logging.config

# change directory to the location where the script is running
from os import chdir

# parse arguments
import sys

# handle importing libraries based on config file
import importlib

# loop delay - sleep
from time import sleep

# clock
from datetime import datetime

##### PyPi Modules #####
# handle http requests
import requests

# rate limit the queries on the LMS server
from ratelimiter import RateLimiter

# lmsquery-fork for managing communications with lms server
import lmsquery

import constants
import epdlib
from library import configuration
from library import signalhandler
from library import cacheart

import waveshare_epd # explicitly import this to make sure that PyInstaller can find it

In [147]:
def do_exit(status=0, message=None):
    if message:
        border = '\n'+'#'*70 + '\n'
        message = border + message + border + '\n***Exiting***'
        print(message)
        
    try:
        sys.exit(status)
    except Exception as e:
        pass

In [6]:
def scan_servers():
    """scan for and list all available LMS Servers and players"""
    print(f'Scanning for available LMS Server and players')
    servers = lmsquery.scanLMS()
    if not servers:
        print('Error: no LMS servers were found on the network. Is there one running?')
        do_exit(1)
    print('servers found:')
    print(servers)
    players = lmsquery.LMSQuery().get_players()
    # print selected keys for each player
    keys = ['name', 'playerid', 'modelname']
    for p in players:
        print('players found:')
        try:
            for key in keys:
                print(f'{key}: {p[key]}')
            print('\n')
        except KeyError as e:
            pass 

In [313]:
def main():
    #### CONSTANTS ####
    # pull the absolute path from the constants file that resides in the root of the project
    absPath = constants.absPath
    # change the working directory - simplifies all the other path work later
    chdir(absPath)
    
    version = constants.version
    app_name = constants.app_name
    app_long_name = constants.app_long_name
    url = constants.url
        
    ## CONFIGURATION FILES ##
    # logging configuration file
    logging_cfg = constants.logging_cfg
    
    # default base configuration file
    default_cfg = constants.default_cfg
    system_cfg = configuration.fullPath(constants.system_cfg)
    user_cfg = configuration.fullPath(constants.user_cfg)
    
    # set the waveshare library
    waveshare = constants.waveshare
    
    # set plugins library
    plugins = constants.plugins
    
    # file containing layouts
    layouts_file = constants.layouts
    
    default_clock = constants.clock
    
    
    # FORMATTERS
    keyError_fmt = 'KeyError: configuration file section [{}] bad/missing value: "{}"'
    configError_fmt = 'configuration file error: see section [{}] in config file: {}'
    moduleError_fmt = 'failed to load module "{}" {}'
    
    
    #### CONFIGURATION ####
    
    ## LOGGING INIT
    logging.config.fileConfig(logging_cfg)
    
    #### COMMANDLINE ARGS ####
    options = configuration.Options(sys.argv)
    # add options to the configuration object
    # options that override the configuration file options, add in the format: 
    # dest=[[ConfigFileSectionName]]__[[Option_Name]]
    #                               ^^ <-- TWO underscores `__`
    # specifying arguments with #ignore_none=True and ignore_false=True will exclude
    # these arguments entirely from the nested dictionary making it easier to merge
    # the command line arguments into the configuration file without adding unwanted options
    # with default values that potentially conflict or overwrite the config files
    
    # set logging level
    options.add_argument('-l', '--log-level', ignore_none=True, metavar='LOG_LEVEL',
                         type=str, choices=['DEBUG', 'INFO', 'WARNING', 'ERROR'], 
                         dest='main__log_level', 
                         help='set logging level: DEBUG, INFO, WARNING, ERROR')

    # alternative user_cfg file -- do not add this to the options dictionary if NONE
    options.add_argument('-c', '--config', type=str, required=False, metavar='/path/to/cfg/file.cfg',
                         dest='user_cfg', ignore_none=True, default=user_cfg,
                         help=f'use the specified configuration file; default user config: {user_cfg}')
    
    # daemon mode
    options.add_argument('-d', '--daemon', required=False,
                         default=False, dest='main__daemon', action='store_true', 
                         help='run in daemon mode (ignore user configuration)')
    
    # list servers 
    options.add_argument('-s', '--list-servers', action='store_true', 
                         dest='list_servers',
                         default=False, 
                         help='list servers and any players found on local network and exit')
    
    # set the player-id on the command line -- do not add if set to NONE
    options.add_argument('-p', '--player-id', type=str, required=False, metavar='playerName',
                         default=False, dest='lms_server__player_id', ignore_none=True,
                         help='set the name of the player to monitor')
    
    # display the version and exit
    options.add_argument('-V', '--version', action='store_true', required=False,
                         dest='version', default=False, 
                         help='display version nubmer and exit')    

    
    # parse the command line options
    options.parse_args()
    
    #### ACTION COMMAND LINE ARGUMENTS ####
    # print version and exit
    if options.options.version:
        print(f'{version}')
        do_exit(0)
    
    # scan for local LMS servers and players, then exit
    if options.options.list_servers:
        scan_servers()
        do_exit(0)
    
    # user a user specified configuration file
    if 'user_cfg' in options.opts_dict:
        user_cfg = options.opts_dict['user_cfg']
    
    # always try to use these two configuration files at launch
    config_file_list = [default_cfg, system_cfg]
    
    # check if running in daemon mode; append user config file
    if not options.options.main__daemon:
        config_file_list.append(user_cfg)
    
    # read all the configuration files in the list - values in left most file is default
    # values in each file to the right override previous values
    config_file = configuration.ConfigFile(config_files=config_file_list)
    
    # merge the configuration file(s) values with command line options
    # command line options override config files
    config = configuration.merge_dict(config_file.config_dict, options.nested_opts_dict)
    
    # kludge to work around f-strings with quotes in Jupyter
    ll = config['main']['log_level']
    logging.root.setLevel(ll)
    logging.debug(f'log level set: {ll}')    
    
    #### HARDWARE INIT ####
    ## EPD INIT ##
    try:
        epd_module = '.'.join([waveshare, config['layouts']['display']])
        epd = importlib.import_module(epd_module)
#         epd = importlib.import_module('.'.join([waveshare, 'foo']))
    except KeyError as e:
        logging.fatal(keyError_fmt.format('layouts', 'display'))
#         do_exit(1, message=f'Bad or missing "display" type in section [layouts] in {config_file_list}')
        do_exit(1, message=keyError_fmt.format('layouts', 'display'))

    except ModuleNotFoundError as e:
        logging.fatal(configError_fmt.format('layouts', config_file_list))
#         do_exit(1, message=f'Bad or missing "display" in section [layouts] in {config_file_list}')
        do_exit(1, message=moduleError_fmt(epd_module, configError_fmt.format('layouts', config_file_list)))
        
    ## SCREEN INIT ##
    screen = epdlib.Screen()
    try:
        screen.epd = epd
    except PermissionError as e:
        logging.critical(f'Error initializing EPD interface: {e}')
        logging.critical('The user executing this program does not have access to the SPI devices.')
        do_exit(0, 'This user does not have access to the SPI group\nThis can typically be resolved by running:\n$ sudo groupadd <username> spi')
    
    screen.initEPD()
      
    
    ## LAYOUT INIT ##
    
    # import layouts
    logging.debug(f'importing layouts from file: {layouts_file}')
    try:
        layouts = importlib.import_module(layouts_file)
        playing_layout_format = getattr(layouts, config['layouts']['now_playing'])
        plugin_layout_format = getattr(layouts, config['layouts']['plugin'])
        splash_layout_format = getattr(layouts, config['layouts']['splash'])
        error_layout_format = getattr(layouts, config['layouts']['error'])
    except ModuleNotFoundError as e: 
        logging.fatal(moduleError_fmt.format('layouts', f'- could not load layouts from module {config_file}'))
        do_exit(1, message=moduleError_fmt.format('layouts', f'- could not load layouts from module {layouts_file}'))
    
    except (KeyError, AttributeError) as e:
        logging.fatal(keyError_fmt.format('layouts', e.args[0]))
        logging.fatal(configError_fmt.format('layotus', config_file_list))
        do_exit(1, message=keyError_fmt.format('layouts', e.args[0]))
    
    
    playing_layout = epdlib.Layout(layout=playing_layout_format, resolution=screen.resolution)
    plugin_layout = epdlib.Layout(layout=plugin_layout_format, resolution=screen.resolution)
    error_layout = epdlib.Layout(layout=error_layout_format, resolution=screen.resolution)
        
    
    ## PLUGIN INIT ##
    try:
        plugin = importlib.import_module('.'.join([plugins, config['modules']['plugin']]))
    except KeyError as e:
        logging.error(keyError_fmt.format('modules', config['modules']['plugin']))
    except ModuleNotFoundError as e:
        logging.error(moduleError_fmt(config['modules']['plugin'], f'- {e.args[0]}'))
        logging.error('falling back to default')
        try:
            plugin = importlib.import_module(default_clock)
        except ModuleNotFoundError as e:
            myE = moduleError_fmt(default_clock, f'- {e.args[0]}')
            logging.fatal(myE)
            do_exit(1, myE)
    
    try:
        plugin_update = int(config['modules']['plugin_update'])
    except KeyError as e:
        myE = keyError_fmt.format('modules', 'plugin_update')
        logging.error(myE)
        logging.error('setting plugin update to 60 seconds')
        pluggin_update = 60
    
    
    #### EXECUTION ####
    logging.debug(f'starting with configuration: {config}')
    
    ## EXEC VARIABLES ##
    # signal handler for catching and handling HUP/KILL signals
    sigHandler = signalhandler.SignalHandler()
    
    # LMS Query rate limiter wrapper - allow max of `max_calls` per `period` (seconds)
    lmsQuery_ratelimit = RateLimiter(max_calls=1, period=3)
    
    # LMS Query Object creation - rate limit to once/30 seconds
    lmsDelay_ratelimit = RateLimiter(max_calls=1, period=30)
    
    # logitech media server interface object
    lms = None
    
    # refresh when true
    refresh = False
    refresh_delay = 60
    
    # vars for managing track ID, mode, album art
    nowplaying_id = None
    nowplaying_mode = "Pause"
    artwork_cache = cacheart.CacheArt(app_long_name)
    
    # check for the word `true` - config file is all stored as type `str`
    if config['main']['splash_screen'].lower() == 'true':
        splash_layout = epdlib.Layout(layout=splash_layout_format, resolution=screen.resolution)
        splash_layout.update_contents({'app_name': app_name,
                                       'version': f'version: {version}',
                                       'url': url})
        refresh = splash_layout
#         screen.elements = splash_layout.blocks.values()
#         screen.concat()
#         screen.writeEPD()
    else:
        pass
        
    
    try:
        while not sigHandler.kill_now:
            response = None
        
            # check of LMS query object is configured
            if lms:
                with lmsQuery_ratelimit:
                    try:
                        myE = f'query lms server for status of player {config["lms_server"]["player_name"]}: {lms.player_id}'
                        logging.debug(myE)
                        response = lms.now_playing()
                    except requests.exceptions.ConnectionError as e:
                        logging.warning(f'server could not find active player_id: {lms.player_id}')
                        logging.warning(f'is the specified player active?')
                        error_layout.update_contents({'message': f'{config["lms_server"]["player_name"]} does not appear to be available. Is it on?', 'time': 'NO PLAYER'})
                        refresh = error_layout
                        response = None
                        
                    except KeyError as e:
                        myE = f'No playlist is active on {config["lms_server"]["player_name"]}'
                        logging.info(myE)
                        response =  {'title': 'No music is queued', 'id': 'NONE', 'mode': 'No Playlist'}
                        nowplaying_mode = response['mode']                 
    
            else: # try to create an lms query object
                with lmsDelay_ratelimit:
                    try:
                        logging.debug('setting up lms query connection')
                        lms = lmsquery.LMSQuery(**config['lms_server'])
                        if not lms.player_id:
                            raise ValueError(keyError_fmt.format('lms_server', 'player_name'))
                        logging.info('lms query connection created')

                    except TypeError as e:
                        logging.critical(configError_fmt.format('lms_server', config_file_list))
                        error_layout.update_contents({'message': configError_fmt.format('lms_server', config_file_list)})
                        refresh = error_layout

                    except ValueError as e:
                        myE = keyError_fmt.format('lms_server', 'player_name')
                        logging.critical(myE)
                        myE = 'LMS QUERY ERROR: \n' + myE
                        error_layout.update_contents({'message': myE, 'time': 'LMS ERROR'})
                        refresh = error_layout
                        
                    except OSError as e:
                        myE = 'could not find LMS servers due to network error '
                        logging.warning(myE)
                        logging.warning('delaying start of LMS query connection')
                        myE = 'LMS QUERY ERROR: ' + myE + e.args[0]
                        error_layout.update_contents({'message': myE, 'time': 'LMS ERROR'})
                        refresh = error_layout
                        

            if response:
                resp_id = response['id']
                resp_mode = response['mode']
                if resp_id != nowplaying_id or resp_mode != nowplaying_mode:
                    logging.info(f'track/mode change to: {resp_mode}')
                    nowplaying_id = resp_id
                    nowplaying_mode = resp_mode
                    
                    # attempt to fetch artwork 
                    try:
                        logging.debug('attempting to download artwork')
                        artwork = artwork_cache.cache_artwork(response['artwork_url'], response['album_id'])
                    except KeyError as e:
                        logging.warning('no artwork available')
                        artwork = None
                    if not artwork:
                        logging.warning(f'using default artwork file: {noartwork}')
                        artwork = noartwork
                    # add the path to the downloaded album art into the response
                    response['coverart'] = str(artwork)
                             
                    # update the layout with the values in the response
                    playing_layout.update_contents(response)
                    
                    # refresh contains the current layout
                    refresh = playing_layout
                    #set delay to 60 seconds
                    refresh_delay = 60
                else:
                    refresh = False
                
            if nowplaying_mode != "play" and screen.update.last_updated > refresh_delay:
                logging.debug(f'next update will be in {refresh_delay} seconds')
                logging.info('music appears to be paused, switching to plugin display')
                update = plugin.get_time()
                update['mode'] = nowplaying_mode
                plugin_layout.update_contents(update)
                refresh = plugin_layout
                refresh_delay = plugin_update                
            
            
            # check if `refresh` has been updated 
            if refresh and isinstance(refresh, epdlib.Layout):
                logging.debug('refresh display')
                screen.initEPD()
                screen.elements = refresh.blocks.values()
                screen.concat()
                screen.writeEPD()
                # set response to 
                refresh = False
            
            # sleep for half a second every cycle
            sleep(0.5)
            
    finally:
        print('Received exit signal - cleaning up')
        
#         screen.initEPD()
#         screen.clearEPD()
        
    return config

In [ ]:
if __name__ == '__main__':
    o = main()

21:53:11 <ipython-input-313-1404d61fc9be>:main:125:DEBUG - log level set: DEBUG
21:53:11 Screen:__init__:133:INFO - Screen created
21:53:12 epd5in83:ReadBusy:69:DEBUG - e-Paper busy
21:53:12 epd5in83:ReadBusy:72:DEBUG - e-Paper busy release
21:53:12 Screen:initEPD:209:INFO - <waveshare_epd.epd5in83.EPD object at 0xad8aea90> initialized
21:53:12 <ipython-input-313-1404d61fc9be>:main:158:DEBUG - importing layouts from file: layouts
21:53:12 Layout:__init__:151:INFO - Layout created
21:53:12 Layout:__init__:157:DEBUG - no default font specified
21:53:12 Layout:layout:270:DEBUG - calculating values from layout for resolution [600, 448]
21:53:12 Layout:_calculate_layout:298:DEBUG - ***title***
21:53:12 Layout:_check_keys:173:DEBUG - checking key/values
21:53:12 Layout:_calculate_layout:305:DEBUG - dimensions: (600, 224)
21:53:12 Layout:_calculate_layout:334:DEBUG - has explict position
21:53:12 Layout:_calculate_layout:336:DEBUG - abs_coordinates: (0, 0)
21:53:12 Layout:_calculate_layout:29

21:53:15 Block:abs_coordinates:160:DEBUG - absolute coordinates: (0, 0)
21:53:15 Block:__init__:386:INFO - TextBlock created
21:53:15 Block:maxchar:457:DEBUG - maximum characters per line: 17
21:53:15 Block:text_formatter:499:DEBUG - formatted list:
 ['.']
21:53:15 Block:_text2image:510:DEBUG - creating blank image area: (600, 384) with inverse: True
21:53:15 Block:_text2image:525:DEBUG - line size: 18, 89
21:53:15 Block:_text2image:529:DEBUG - max x dim so far: 18
21:53:15 Block:_text2image:533:DEBUG - dimensions of text portion of image: (18, 89)
21:53:15 Block:_text2image:543:DEBUG - drawing text at 0, 0
21:53:15 Block:_text2image:544:DEBUG - with dimensions: 18, 89
21:53:15 Block:_text2image:554:DEBUG - randomly positioning text within area
21:53:15 Block:_text2image:570:DEBUG - pasting text portion at coordinates: 37, 14
21:53:15 Layout:_set_images:362:DEBUG - ***time***)
21:53:15 Layout:_set_images:366:DEBUG - set text block
21:53:15 Block:area:142:DEBUG - block area: (300, 64)
2

21:53:18 Layout:_set_images:366:DEBUG - set text block
21:53:18 Block:area:142:DEBUG - block area: (600, 269)
21:53:18 Block:inverse:117:DEBUG - set inverse: False
21:53:18 Block:abs_coordinates:160:DEBUG - absolute coordinates: (0, 0)
21:53:18 Block:__init__:386:INFO - TextBlock created
21:53:19 Block:maxchar:457:DEBUG - maximum characters per line: 8
21:53:19 Block:text_formatter:499:DEBUG - formatted list:
 ['.']
21:53:19 Block:_text2image:510:DEBUG - creating blank image area: (600, 269) with inverse: False
21:53:19 Block:_text2image:525:DEBUG - line size: 38, 188
21:53:19 Block:_text2image:529:DEBUG - max x dim so far: 38
21:53:19 Block:_text2image:533:DEBUG - dimensions of text portion of image: (38, 188)
21:53:19 Block:_text2image:541:DEBUG - hcenter line: .
21:53:19 Block:_text2image:543:DEBUG - drawing text at 0, 0
21:53:19 Block:_text2image:544:DEBUG - with dimensions: 38, 188
21:53:19 Block:_text2image:570:DEBUG - pasting text portion at coordinates: 281, 40
21:53:19 Layout:

21:53:25 epd5in83:ReadBusy:72:DEBUG - e-Paper busy release
21:53:25 Screen:initEPD:209:INFO - <waveshare_epd.epd5in83.EPD object at 0xad8aea90> initialized
21:53:25 Screen:concat:189:DEBUG - pasting image at: (0, 0)
21:53:25 Screen:concat:189:DEBUG - pasting image at: (0, 384)
21:53:25 Screen:concat:189:DEBUG - pasting image at: (300, 384)
21:53:25 epd5in83:getbuffer:128:DEBUG - imwidth = 600  imheight =  448 
21:53:30 epd5in83:ReadBusy:69:DEBUG - e-Paper busy
21:53:34 epd5in83:ReadBusy:72:DEBUG - e-Paper busy release
21:53:34 epd5in83:ReadBusy:69:DEBUG - e-Paper busy
21:53:34 epd5in83:ReadBusy:72:DEBUG - e-Paper busy release
21:53:34 epdconfig:module_exit:86:DEBUG - spi end
21:53:34 epdconfig:module_exit:89:DEBUG - close 5V, Module enters 0 power consumption ...
21:53:34 <ipython-input-313-1404d61fc9be>:main:252:DEBUG - query lms server for status of player slimpi: dc:a6:32:29:99:f0
21:53:34 <ipython-input-313-1404d61fc9be>:main:301:INFO - track/mode change to: stop
21:53:34 <ipython-

21:54:38 wordclock:get_time:116:DEBUG - using 21:54
21:54:38 Layout:update_contents:394:INFO - updating blocks
21:54:38 Layout:update_contents:400:DEBUG - updating block: wordtime
21:54:38 Block:text_formatter:499:DEBUG - formatted list:
 ['It is almost Ten', "'Til Ten"]
21:54:38 Block:_text2image:510:DEBUG - creating blank image area: (600, 384) with inverse: True
21:54:38 Block:_text2image:525:DEBUG - line size: 476, 90
21:54:38 Block:_text2image:529:DEBUG - max x dim so far: 476
21:54:38 Block:_text2image:525:DEBUG - line size: 212, 90
21:54:38 Block:_text2image:533:DEBUG - dimensions of text portion of image: (476, 180)
21:54:38 Block:_text2image:543:DEBUG - drawing text at 0, 0
21:54:38 Block:_text2image:544:DEBUG - with dimensions: 476, 90
21:54:38 Block:_text2image:543:DEBUG - drawing text at 0, 90
21:54:38 Block:_text2image:544:DEBUG - with dimensions: 212, 90
21:54:38 Block:_text2image:554:DEBUG - randomly positioning text within area
21:54:38 Block:_text2image:570:DEBUG - pas

...

21:56:49 epdconfig:module_exit:89:DEBUG - close 5V, Module enters 0 power consumption ...
21:56:53 <ipython-input-313-1404d61fc9be>:main:252:DEBUG - query lms server for status of player slimpi: dc:a6:32:29:99:f0
21:56:56 <ipython-input-313-1404d61fc9be>:main:252:DEBUG - query lms server for status of player slimpi: dc:a6:32:29:99:f0
21:57:00 <ipython-input-313-1404d61fc9be>:main:252:DEBUG - query lms server for status of player slimpi: dc:a6:32:29:99:f0
21:57:03 <ipython-input-313-1404d61fc9be>:main:252:DEBUG - query lms server for status of player slimpi: dc:a6:32:29:99:f0
21:57:03 <ipython-input-313-1404d61fc9be>:main:301:INFO - track/mode change to: pause
21:57:03 <ipython-input-313-1404d61fc9be>:main:307:DEBUG - attempting to download artwork
21:57:03 cacheart:cache_artwork:51:DEBUG - attempting to cache artwork
21:57:03 cacheart:cache_artwork:60:DEBUG - artwork previously cached
21:57:03 Layout:update_contents:394:INFO - updating blocks
21:57:03 Layout:update_contents:403:DEBUG -

21:58:07 Block:_text2image:525:DEBUG - line size: 246, 90
21:58:07 Block:_text2image:533:DEBUG - dimensions of text portion of image: (481, 180)
21:58:07 Block:_text2image:543:DEBUG - drawing text at 0, 0
21:58:07 Block:_text2image:544:DEBUG - with dimensions: 481, 90
21:58:07 Block:_text2image:543:DEBUG - drawing text at 0, 90
21:58:07 Block:_text2image:544:DEBUG - with dimensions: 246, 90
21:58:07 Block:_text2image:554:DEBUG - randomly positioning text within area
21:58:07 Block:_text2image:570:DEBUG - pasting text portion at coordinates: 0, 178
21:58:07 Layout:update_contents:400:DEBUG - updating block: time
21:58:07 Block:text_formatter:499:DEBUG - formatted list:
 ['21:58']
21:58:07 Block:_text2image:510:DEBUG - creating blank image area: (300, 64) with inverse: True
21:58:07 Block:_text2image:525:DEBUG - line size: 79, 44
21:58:07 Block:_text2image:529:DEBUG - max x dim so far: 79
21:58:07 Block:_text2image:533:DEBUG - dimensions of text portion of image: (79, 44)
21:58:07 Block:

21:58:33 <ipython-input-313-1404d61fc9be>:main:301:INFO - track/mode change to: stop
21:58:33 <ipython-input-313-1404d61fc9be>:main:307:DEBUG - attempting to download artwork
21:58:33 cacheart:cache_artwork:51:DEBUG - attempting to cache artwork
21:58:33 cacheart:cache_artwork:65:DEBUG - artwork url: http://192.168.178.9:9000/music/6031b307/cover.jpg
21:58:33 cacheart:cache_artwork:71:DEBUG - writing artwork file: /tmp/com.txoof.slimpi/5663.jpg
21:58:33 cacheart:cache_artwork:75:DEBUG - wrote ablum artwork to: /tmp/com.txoof.slimpi/5663.jpg
21:58:33 Layout:update_contents:394:INFO - updating blocks
21:58:33 Layout:update_contents:403:DEBUG - ignoring block id
21:58:33 Layout:update_contents:400:DEBUG - updating block: title
21:58:33 Block:text_formatter:499:DEBUG - formatted list:
 ['Full Force Gale']
21:58:33 Block:_text2image:510:DEBUG - creating blank image area: (600, 224) with inverse: False
21:58:33 Block:_text2image:525:DEBUG - line size: 393, 78
21:58:33 Block:_text2image:529:D

21:59:37 Block:_text2image:544:DEBUG - with dimensions: 340, 90
21:59:37 Block:_text2image:543:DEBUG - drawing text at 0, 90
21:59:37 Block:_text2image:544:DEBUG - with dimensions: 396, 98
21:59:37 Block:_text2image:543:DEBUG - drawing text at 0, 188
21:59:37 Block:_text2image:544:DEBUG - with dimensions: 173, 90
21:59:37 Block:_text2image:554:DEBUG - randomly positioning text within area
21:59:37 Block:_text2image:570:DEBUG - pasting text portion at coordinates: 10, 80
21:59:37 Layout:update_contents:400:DEBUG - updating block: time
21:59:37 Block:text_formatter:499:DEBUG - formatted list:
 ['21:59']
21:59:37 Block:_text2image:510:DEBUG - creating blank image area: (300, 64) with inverse: True
21:59:37 Block:_text2image:525:DEBUG - line size: 79, 44
21:59:37 Block:_text2image:529:DEBUG - max x dim so far: 79
21:59:37 Block:_text2image:533:DEBUG - dimensions of text portion of image: (79, 44)
21:59:37 Block:_text2image:543:DEBUG - drawing text at 0, 0
21:59:37 Block:_text2image:544:DEB

22:00:08 epd5in83:getbuffer:128:DEBUG - imwidth = 600  imheight =  448 
22:00:14 epd5in83:ReadBusy:69:DEBUG - e-Paper busy
22:00:17 epd5in83:ReadBusy:72:DEBUG - e-Paper busy release
22:00:17 epd5in83:ReadBusy:69:DEBUG - e-Paper busy
22:00:17 epd5in83:ReadBusy:72:DEBUG - e-Paper busy release
22:00:17 epdconfig:module_exit:86:DEBUG - spi end
22:00:17 epdconfig:module_exit:89:DEBUG - close 5V, Module enters 0 power consumption ...
22:00:21 <ipython-input-313-1404d61fc9be>:main:252:DEBUG - query lms server for status of player slimpi: dc:a6:32:29:99:f0
22:00:24 <ipython-input-313-1404d61fc9be>:main:252:DEBUG - query lms server for status of player slimpi: dc:a6:32:29:99:f0
22:00:24 <ipython-input-313-1404d61fc9be>:main:301:INFO - track/mode change to: pause
22:00:24 <ipython-input-313-1404d61fc9be>:main:307:DEBUG - attempting to download artwork
22:00:24 cacheart:cache_artwork:51:DEBUG - attempting to cache artwork
22:00:24 cacheart:cache_artwork:60:DEBUG - artwork previously cached
22:00:

In [275]:
print(o)

{'main': {'splash_screen': 'True', 'log_level': 'DEBUG', 'daemon': False}, 'lms_server': {'host': '', 'port': '9000', 'player_id': '', 'player_name': ''}, 'layouts': {'display': 'epd5in83', 'now_playing': 'threeRow', 'stopped': 'wordclock', 'splash': 'splash', 'error': 'error'}, 'modules': {'clock': 'wordclock', 'clock_update': '295'}, '__cmd_line': {'user_cfg': PosixPath('/home/pi/.config/com.txoof.slimpi/slimpi.cfg'), 'list_servers': False, 'version': False}}


In [229]:
dir(o)

['__cause__',
 '__class__',
 '__context__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__suppress_context__',
 '__traceback__',
 'args',
 'with_traceback']

In [186]:
o.parser.getboolean('main', 'splash_screen')

False